In [2]:
import os
import sys

sys.path.append(os.path.abspath(".."))

In [10]:
import requests
from project_config import get_project_config
import os

config = get_project_config()

# Replace with your actual Google Places API key
os.environ['GOOGLE_PLACESAPI_KEY'] = config.GOOGLE_PLACES_API_KEY


In [12]:

def get_place_id(place_query):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": place_query,
        "inputtype": "textquery",
        "fields": "place_id",
        "key": config.GOOGLE_PLACES_API_KEY
    }
    response = requests.get(url, params=params)
    print(response.json())
    data = response.json()

    if data["status"] == "OK" and data["candidates"]:
        return data["candidates"][0]["place_id"]
    else:
        print("Place not found.")
        return None

def get_place_details(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,formatted_address,opening_hours,website,rating,user_ratings_total",
        "key": config.GOOGLE_PLACES_API_KEY
    }
    response = requests.get(url, params=params)
    return response.json()

def verify_place_from_web_result(web_result):
    print(f"Searching for: {web_result}")
    place_id = get_place_id(web_result)

    if not place_id:
        return {"exists": False, "details": None}

    details = get_place_details(place_id)
    result = details.get("result", {})

    return {
        "exists": True,
        "name": result.get("name"),
        "address": result.get("formatted_address"),
        "is_open_now": result.get("opening_hours", {}).get("open_now"),
        "website": result.get("website"),
        "rating": result.get("rating"),
        "user_ratings_total": result.get("user_ratings_total")
    }

# Example from Tavily/web search
web_result_text = "Le Vantage Cafe Bar, Road No 13, Banjara Hills, Hyderabad, Telangana, India, 500034"
info = verify_place_from_web_result(web_result_text)

print("Verification Result:")
print(info)


Searching for: Le Vantage Cafe Bar, Road No 13, Banjara Hills, Hyderabad, Telangana, India, 500034
{'candidates': [{'place_id': 'ChIJSzs3-TGRyzsR7cuQ0lDJpBI'}], 'status': 'OK'}
Verification Result:
{'exists': True, 'name': 'Lé Vantage Café Bar', 'address': '195/A, Rd Number 13, Jubilee Hills, Hyderabad, Telangana 500033, India', 'is_open_now': True, 'website': 'http://www.levantagecafebar.com/', 'rating': 4.3, 'user_ratings_total': 7225}
